In [1]:
import numpy as np 
from scipy.integrate import odeint
import os, sys, warnings
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import utility as ut
import surrogate_nn as srnn
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
warnings.filterwarnings('ignore')

In [2]:
DTYPE = 'float64'
L63_data_path = '../data/L63-trajectories'
save_folder='../data/adaptive-rate'
log_interval = 100
milestones = [10*2**n for n in range(15)]
learning_rate = 1e-3
drop = 0.7
steps = int(1e5)
save_interval = 100
N = 2000
L0 = 0.4
L1 = 3.5
beta = 4e-5
train = np.load(f'{L63_data_path}/train.npy').astype(DTYPE)
test = np.load(f'{L63_data_path}/test.npy')[:, :, :1000].astype(DTYPE)

model = srnn.SurrogateModel_NN(3, 100, name='nn', save_folder=save_folder)
model.learn(train[:, :N], steps, 1e-4, batch_size='GD', log_interval=10, save_interval=100,\
            update_interval=100, del_loss=0.01, fluc_lim=0.01, del_rate=0.01)
tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:100], error_threshold=0.05)
tau_f_1 = tau_f_se.mean()
tau_f_1

step: 0 loss: 1573993.147849 time elapsed: 0.0024 learning rate: 0.000100, scenario: 0, slope: None, fluctuations: 0
step: 10 loss: 1562485.526607 time elapsed: 0.0163 learning rate: 0.000100, scenario: 0, slope: None, fluctuations: 0
step: 20 loss: 1551064.813645 time elapsed: 0.0272 learning rate: 0.000100, scenario: 0, slope: None, fluctuations: 0
step: 30 loss: 1539738.344035 time elapsed: 0.0381 learning rate: 0.000100, scenario: 0, slope: None, fluctuations: 0
step: 40 loss: 1528508.663452 time elapsed: 0.0490 learning rate: 0.000100, scenario: 0, slope: None, fluctuations: 0
step: 50 loss: 1517374.874523 time elapsed: 0.0597 learning rate: 0.000100, scenario: 0, slope: None, fluctuations: 0
step: 60 loss: 1506333.827075 time elapsed: 0.0714 learning rate: 0.000100, scenario: 0, slope: None, fluctuations: 0
step: 70 loss: 1495381.468050 time elapsed: 0.0834 learning rate: 0.000100, scenario: 0, slope: None, fluctuations: 0
step: 80 loss: 1484513.660793 time elapsed: 0.0945 learni

0.962416

In [3]:
from scipy import stats
x = [15217365, 1918240., 2192992, 1542356.749509, 1e6, 1195660.183959, 854033.622230]
y = np.arange(len(x))
res = stats.linregress(y, x)

In [4]:
res.slope

-1633148.0630497143

In [5]:
from scipy.signal import argrelextrema
argrelextrema(np.array(x), np.greater)[0]

array([2, 5])